In [1]:
"""

This script creates weights to redistribute the melt of each basin to the front of the ice shelves of the basins (instead of redistributing uniformly in front of the whole basin)
Step 1: Define the ice-shelf fronts (by regridding BedMachine to your native grid)
Step 2: Redistribute the melt at the ice-shelf fronts of given basins

Author: Clara Burgard

"""

'\n\nThis script creates weights to redistribute the melt of each basin to the front of the ice shelves of the basins (instead of redistributing uniformly in front of the whole basin)\nStep 1: Define the ice-shelf fronts\nStep 2: Read in the minimum and maximum injection depths and compare to the local bathymetry at the ice-shelf fronts. Correct when local bathymetry is shallower than the maximum injection depths.\n\nAuthor: Clara Burgard\n\n'

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
import subprocess, sys

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [13]:
inputpath = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/Schmidt_stuff/'
path4 = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/' 
inputpath_BedMachine='/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/' # download BedMachine v3 from here: https://nsidc.org/data/nsidc-0756/versions/3

In [14]:
your_model_gridfile = path4+'grid_05degrees.txt'

In [15]:
#### PREPARE MASKS FOR YOUR MODEL GRID
# Regrid BedMachine ice shelf/grounded ice/ocean mask to your model grid
subprocess.run('cdo remapnn,'+your_model_gridfile+' -selname,mask '+inputpath_BedMachine+'BedMachineAntarctica-v3.nc '+inputpath+'BedMachine3_mask_onmodelgrid.nc', shell = True, executable="/bin/bash")

cdo    remapnn: Nearest neighbor weights from curvilinear (13333x13333) to lonlat (720x360) grid


CompletedProcess(args='cdo remapnn,/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/grid_05degrees.txt -selname,mask_region_extrapolated_to_ocean_2D /data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/Schmidt_stuff/IMBIE_mask_basins_extrapolated_to_ocean.nc /data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/Schmidt_stuff/IMBIE_mask_basins_extrapolated_to_ocean_onmodelgrid.nc', returncode=0)

In [18]:
# Regrid the IMBIE basin mask to your model grid
subprocess.run('cdo remapnn,'+your_model_gridfile+' -selname,mask_region_extrapolated_to_ocean_2D '+inputpath+'IMBIE_mask_basins_extrapolated_to_ocean.nc '+inputpath+'IMBIE_mask_basins_extrapolated_to_ocean_onmodelgrid.nc',shell = True, executable="/bin/bash")

Warning (cdfScanVarAttr): NetCDF: Variable not found - lat_bnds
Warning (cdfScanVarAttr): NetCDF: Variable not found - lon_bnds


cdo    remapnn: Nearest neighbor weights from curvilinear (3041x3041) to lonlat (720x360) grid


CompletedProcess(args='cdo remapnn,/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/grid_05degrees.txt -selname,mask_region_extrapolated_to_ocean_2D /data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/Schmidt_stuff/IMBIE_mask_basins_extrapolated_to_ocean.nc /data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/Schmidt_stuff/IMBIE_mask_basins_extrapolated_to_ocean_onmodelgrid.nc', returncode=0)

In [36]:
#### READ IN DATA
IMBIE_mask = xr.open_dataset(inputpath  + 'IMBIE_mask_basins_extrapolated_to_ocean_onmodelgrid.nc')
BedMachine_mask = xr.open_dataset(inputpath  + 'BedMachine3_mask_onmodelgrid.nc')
# set Lake Vostok and non-ice covered ground point to grounded ice
BedMachine_mask_clean = BedMachine_mask['mask'].where(BedMachine_mask['mask'] != 4,2).where(BedMachine_mask['mask']!= 1,2) 
# prescribed melt per basin
melt_file = xr.open_dataset(inputpath  + 'AQ_subshelf_melt.nc')
# cell area of your model grid
cell_area = xr.open_dataset(inputpath + 'cellarea_05degrees.nc')['cell_area'] 


In [ ]:
# not needed yet
bathy = model['bathy_metry']
isfmask = (BedMachine_mask_clean == 3)

In [22]:
#### IDENTIFY THE ICE SHELF FRONTS
# Create a mask discriminating between land, ocean and ice shelf
mask_0_1_2 = BedMachine_mask_clean.copy()
mask_0_1_2 = mask_0_1_2.where(BedMachine_mask_clean != 2, 400) # land
mask_0_1_2 = mask_0_1_2.where(BedMachine_mask_clean != 0, 0) # ocean
mask_0_1_2 = mask_0_1_2.where(BedMachine_mask_clean != 3, 200) # ice shelf

# set all ice shelves to 300
mask_front0 = mask_0_1_2.where((mask_0_1_2 == 0) | (mask_0_1_2 == 400), 300).copy()

mask_front = mask_front0.copy()
# check all directions and set points at border between ocean and ice shelf (300-0) to 500
# CAREFUL: replace lon and lat by the name of your horizontal coordinates 
mask_front = mask_front.where((mask_front0.shift(lon=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(lon=1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(lat=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(lat=1)-mask_front0)!=300,500)
# cut out all front points
mask_front = mask_front.where(mask_front==500)
# set the ice shelf number
mask_front = mask_front.where(mask_front!=500,IMBIE_mask['mask_region_extrapolated_to_ocean_2D'])

In [63]:
yearinsec = 86400.0 * 365.2422 # number of seconds in a year

### DISTRIBUTE THE MELT
melt_all = melt_file['baseline'] + melt_file['anomaly'] # melt in Gt/yr
melt_kg_per_s = melt_all / yearinsec *10**(12) # melt in kg per s

melt_front = mask_front.copy()
# redistribute the melt uniformly over the front of the ice shelves in the given basin
for id_basin in range(1,19):  
    cell_area_bb = cell_area.where(mask_front == id_basin)
    # melt in kg per s per m^2 
    melt_front = melt_front.where(mask_front != id_basin, melt_kg_per_s.sel(region=id_basin) / cell_area_bb.sum(['lon','lat'])).drop('region') 

In [65]:
melt_ds = xr.Dataset()
melt_ds['melt_to_be_injected'] = melt_front
melt_ds['melt_to_be_injected'].attrs['standard_name'] = 'Sub-shelf melt to be injected between minimal and maximal injection depth given in AQ_subshelf_melt.nc'
melt_ds['melt_to_be_injected'].attrs['units'] = 'kg s^-1 m^-2'

In [66]:
melt_ds

<xarray.Dataset>
Dimensions:              (lon: 720, lat: 360, time: 31)
Coordinates:
  * lon                  (lon) float64 -179.8 -179.2 -178.8 ... 179.2 179.8
  * lat                  (lat) float64 -89.75 -89.25 -88.75 ... 89.25 89.75
  * time                 (time) datetime64[ns] 1991-07-01 ... 2021-07-01
Data variables:
    melt_to_be_injected  (lat, lon, time) float64 nan nan nan ... nan nan nan